# Load environment

In [2]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(override=True)



True

In [14]:
import pickle

def pickleSave(object, name, folder='.', silent=False):

    filename = folder + '/' + name + '.pkl'
    if silent==False:
        print('Saving object {} to pickle file {}'.format(name, filename))
    with open(filename, mode='wb') as fipkl:    
        pickle.dump(object, fipkl)
        
def pickleLoad(name, folder: str, silent=False):
    
    filename = folder + '/' + name + '.pkl'
    if silent==False:
        print('Loading object {} from pickle file {}'.format(name, filename))
    
    try:
        with open(filename, mode='rb') as fipkl:
            myObject=pickle.load(fipkl)
        return myObject
    except IOError:
        print('Pickle file {} not found, returning None object'.format(filename))
        return None

# Additional private data  

!!! This section only need to be run once because it will store embeddings of the scraped webiste in a persistent instance of ChromaDb  

## Installing persistent storage ChromaDB for our embeddings  

https://howaibuildthis.substack.com/p/a-guide-to-installing-chromadb-on  

How to install ChromaDB:  

git clone https://github.com/chroma-core/chroma  

cd chroma  

docker-compose up -d --build  






## Setting up tiktoken 

... for couting the number of tokens in a string of text  

In [3]:
import tiktoken

def concatenate_page_contents(input_documents, separator):
    page_contents = [doc.dict()['page_content'] for doc in input_documents]
    return separator.join(page_contents)

def num_tokens_from_string(string: str, model_name: str = 'gpt-3.5-turbo') -> int:
    encoding = tiktoken.encoding_for_model(model_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

## Loader: Scraping website

In [7]:
from langchain.document_loaders.base import Document
from langchain.utilities import ApifyWrapper

def getScrapingLoader(url):
    loader = ApifyWrapper().call_actor(
        actor_id='apify/website-content-crawler',
        run_input={'startUrls': [{'url': url}]},
        dataset_mapping_function=lambda item: Document(
            page_content=item['text'] or '', 
            metadata={'source': item['url']}
        ),
    )
    return loader

loader = getScrapingLoader(url = 'https://www.steinel.com.au')





## Using loader

This will:  
  
0. setup chroma client_settings (where is my chroma instance, here assuming it runs on a container)  
1. scrape the website into 'documents'  
2. split those 'documents' into chunks of text  
3. create embeddings from those chunks (using default openai embedding text-davinci-003) and store them in Chroma collection LANTERNPAY_COLLECTION

In [9]:
# information about our instance of Chroma

from chromadb.config import Settings
from chromadb import Client

client_settings = Settings(
        chroma_api_impl="rest",
        chroma_server_host="host.docker.internal",  # when you run this inside a devcontainer you need to explicitely say host.docker.internal to signify "devcontainer host localhost"
        chroma_server_http_port="8000"
    )

# about host.docker.internal : https://stackoverflow.com/questions/24319662/from-inside-of-a-docker-container-how-do-i-connect-to-the-localhost-of-the-mach


In [8]:
# loading data "manually" same as doing VectorstoreIndexCreator().from_loaders([loader])

from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

scraped_documents = loader.load()

### Pickling scraped documents

In [15]:
pickleSave(scraped_documents, 'steinel_scraped_documents', folder='.', silent=False)

Saving object steinel_scraped_documents to pickle file ./steinel_scraped_documents.pkl


In [ ]:
scraped_documents = pickleLoad(name='steinel_scraped_documents', folder='.', silent=False)

In [17]:
len(scraped_documents)

171

In [22]:
from pprint import pprint
pprint(scraped_documents[10].page_content)
pprint(scraped_documents[10].metadata['source'])
pprint(scraped_documents[11].page_content)
pprint(scraped_documents[11].metadata['source'])
pprint(scraped_documents[111].page_content)
pprint(scraped_documents[111].metadata['source'])
print('Total number of pages scraped:', len(scraped_documents))
for doc in scraped_documents:
    pprint(doc.metadata['source'])
    print('Page content text length:', len(doc.page_content))
    print('Number of tokens:', num_tokens_from_string(doc.page_content))


('The ultrasonic waves are actively emitted from the sensors and completely '
 'fill the space they cover. Highly sensitive and able to detect minor '
 'movements, the ultrasonic sensors are effective for low motion activities, '
 'which is why they are particularly recommended for rooms in which sedentary '
 'activities are performed.\n'
 'Ultrasonic sensors also detect a larger area than a PIR sensor, meaning '
 'fewer sensors would be required. \n'
 'However, they cannot be mounted higher than 4.2 metres with the sensors '
 'effective range declining proportionally to partition height. They are very '
 'susceptible to high levels of vibration of airflow and not adjustable so not '
 'ideal for areas that require selective coverage.')
'https://www.steinel.com.au/product-category/ultrasonic-sensors/'
('The STEINEL indoor sensorlights combine state-of-the-art sensor technology '
 'with sleek design. Integrated sensors are invisibly concealed inside these '
 'lights to instantly respond 

In [11]:

splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=250)
texts = splitter.split_documents(scraped_documents)

In [23]:
for t in texts[1:5]:
    print('metadata:', t.metadata['source'])
    print('text length:', len(t.page_content))
    print('num_token:', num_tokens_from_string(t.page_content))

metadata: https://www.steinel.com.au/latest-news/
text length: 952
num_token: 221
metadata: https://www.steinel.com.au/contact-us/
text length: 134
num_token: 39
metadata: https://www.steinel.com.au/about-us/
text length: 679
num_token: 121
metadata: https://www.steinel.com.au/product/solar-light-xsolar-l2-s-house-number-silver/
text length: 858
num_token: 183


In [24]:
db = Chroma.from_documents(texts, client_settings=client_settings, embedding = OpenAIEmbeddings(), collection_name='STEINEL_COLLECTION')
retriever = db.as_retriever()

### Problem max query size exceeded from Chroma

clickhouse_connect.driver.httpclient Code: 62. DB::Exception: Syntax error: failed at position 262129 ('0.02478806984617376') (line 108, col 14488): 0.02478806984617376, 0.010010567241768666, -0.016834096559357397, 0.015376775138473653, 0.005069977476240352, -0.022758718007165845, -0.05262698100664378, -0.01. Max query size exceeded: '0.02478806984617376'. (SYNTAX_ERROR) (version 22.9.7.34 (official build)

(found in docker logs for Chroma)

Solution: https://github.com/chroma-core/chroma/issues/687  


# Part 2. Once website has been scraped and store in Chroma DB  



In [25]:
# check chroma db works (you always have to provide the embedding function)

from langchain.embeddings import OpenAIEmbeddings
chromaClient = Client(client_settings)
coll = chromaClient.get_collection(name='STEINEL_COLLECTION', embedding_function=OpenAIEmbeddings())


We used the Chroma client to get our collection. Now we can view the collection name and also find out what has been stored 
in the database.



In [26]:
coll.dict()

{'name': 'STEINEL_COLLECTION',
 'id': UUID('9345a1cc-0479-4614-adef-1f2df41f49e9'),
 'metadata': None}

In [27]:
coll.peek()

{'ids': ['d8a094e6-2901-11ee-a2b0-0242ac110002',
  'd8a09afe-2901-11ee-a2b0-0242ac110002',
  'd8a09c16-2901-11ee-a2b0-0242ac110002',
  'd8a09c98-2901-11ee-a2b0-0242ac110002',
  'd8a09cfc-2901-11ee-a2b0-0242ac110002',
  'd8a09d60-2901-11ee-a2b0-0242ac110002',
  'd8a09e5a-2901-11ee-a2b0-0242ac110002',
  'd8a09f0e-2901-11ee-a2b0-0242ac110002',
  'd8a09f86-2901-11ee-a2b0-0242ac110002',
  'd8a09fea-2901-11ee-a2b0-0242ac110002'],
 'embeddings': [[0.0060437328388491685,
   -0.00869814243142026,
   -0.013389502179035232,
   -0.023919892384897667,
   0.004553773130650533,
   0.01500698137248337,
   -0.019705052230800768,
   -0.032027420752923016,
   -0.008839085069397215,
   -0.004040341031285222,
   0.006694751089517616,
   0.011993503608606165,
   -0.006516895480032119,
   -0.005520233607768919,
   -0.015302287938519522,
   -0.003328918826173886,
   0.017127823670936668,
   0.018456707409051077,
   0.03575903299676724,
   -0.016188210430595793,
   -0.014000251437182625,
   0.01834932218723145

# Loading from persistent vectorstore ChromaDB

In [36]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from chromadb.config import Settings

client_settings = Settings(
        chroma_api_impl="rest",
        chroma_server_host="host.docker.internal",  # when you run this inside a devcontainer you need to explicitely say host.docker.internal to signify "devcontainer host localhost"
        chroma_server_http_port="8000"
    )

chromaDb = Chroma(collection_name='STEINEL_COLLECTION', 
                     embedding_function=OpenAIEmbeddings(),
                     client_settings=client_settings)



## Creating a RetrievalQA chain

This type of chain is used to find information using a retriever.  
A retriever is an interface that returns relevant documents given an unstructured query.  
It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) it.  
Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.  

Here our retriever is going to be made using langchain chromaDb.as_retriever(search_kwargs={"k": 1})  



In [35]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo')

retriever = chromaDb.as_retriever(search_kwargs={"k": 4})

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type='stuff',
    retriever=retriever,
    # chain_type_kwargs={"prompt": PROMPT}, 
    # return_source_documents=True,
    verbose=True
)

# note: the search kwargs is to limit what is sent to the llm, i.e. to change the number of document to return from retriever
# retriever.search_kwargs['k'] = 10

In [34]:
from pprint import pprint

query = "What is this Steinel website about?"
# result = qa.run(query)
result = qa({"query": query})

pprint(result)

# InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 10515 tokens. Please reduce the length of the messages.

Inputs: {'query': 'What is this Steinel website about?'}


> Entering new RetrievalQA chain...
Inputs: {'input_documents': [Document(page_content='© Steinel Australia 2016. All right reserved. \nDesigned By', metadata={'source': 'https://www.steinel.com.au/2180wall/'}), Document(page_content='© Steinel Australia 2016. All right reserved. \nDesigned By', metadata={'source': 'https://www.steinel.com.au/support/'}), Document(page_content='© Steinel Australia 2016. All right reserved. \nDesigned By', metadata={'source': 'https://www.steinel.com.au/daliplus/'}), Document(page_content='© Steinel Australia 2016. All right reserved. \nDesigned By', metadata={'source': 'https://www.steinel.com.au/parking/'})], 'question': 'What is this Steinel website about?'}
Inputs: {'question': 'What is this Steinel website about?', 'context': '© Steinel Australia 2016. All right reserved. \nDesigned By\n\n© Steinel Australia 2016. All right reserved. \nDesigned By\n\n© Steinel Australia 2016. All right rese

In [31]:
query = "Do you have sensors that detects movements in corridor?"
# result = qa.run(query)
result = qa.run(query)

pprint(result)

Inputs: {'query': 'Do you have sensors that detects movements in corridor?'}


> Entering new RetrievalQA chain...
Inputs: {'input_documents': [Document(page_content='Presence Detector Single US KNX\nThe ideal sensor for small corridors. Single US KNX ultrasonic motion detector, ideal for corridors, thoroughfares and stairwells as well as WC facilities, washrooms and store rooms Detection within 10 m in one direction, both radially and tangentially, envelops objects as opposed to penetrating them, detection regardless of temperature, electronically adjustable reach, with digital BUS interface for controlling 4 lighting channels and HVAC.\nVersion\nAbout Presence Detector Single US KNX\nThe ideal sensor for smaller corridors.\nSimple tasks demand simple answers. This is demonstrated by our Single US 180° ultrasonic sensor. Correctly aligned, it detects any movement in corridors and passageways of up to 10 metres in length – even it they take place behind objects. Movements in adjacent r

In [98]:
query = "What is the api endpoint I need to call to submit an invoice?"
# result = qa.run(query)
documents = retriever.get_relevant_documents(query)
result = qa.combine_documents_chain(inputs={'question': query, 'input_documents': documents})

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 41092 tokens. Please reduce the length of the messages.

In [52]:
print(documents[0].dict().keys())

dict_keys(['page_content', 'metadata'])


In [99]:
pprint(result)

('To send an invoice to the TAC (Transport Accident Commission) program, you '
 'will need to integrate with LanternPay by HICAPS. The LanternPay API allows '
 'you to create and send invoices to various programs, including TAC.\n'
 '\n'
 'To get started, you can visit the LanternPay by HICAPS Developer Hub. Here, '
 "you'll find information and API references to help you with the integration "
 'process. If you have any specific questions or need assistance, you can '
 'email integrations@hicaps.com.au and one of their team members will assist '
 'you.\n'
 '\n'
 'Please note that you may need to become an integration partner and request '
 'access to the sandbox environment before you can start sending invoices to '
 'the TAC program.')


### How to deal with problem of maximum context length?

#### Option 1. If you want to keep the same LLM    

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 82282 tokens.  (with query "What is the api endpoint I need to call to submit an invoice?")

1. Find out what is the text sent to the model  
    a. the user query  
    b. documents retrieved from the store
    c. prompt template
2. calculate number of tokens for that text

https://stackoverflow.com/questions/75804599/openai-api-how-do-i-count-tokens-before-i-send-an-api-request  



In [100]:
# context sent to LLM is made of:
# documents retrieved from store: input_document
# user query: query
# prompt template: qa.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template

prompt_template=qa.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template

print('Prompt template:', prompt_template)
print('User query:', query)
print('Number of relevant documents:', len(documents))
print('Start of first document:', documents[0].page_content[0:50], documents[0].metadata['source'])
# print('Start of second document:', documents[1].page_content[0:50], documents[1].metadata['source'])
# print('Start of 3rd document:', documents[2].page_content[0:50], documents[2].metadata['source'])
# print('Start of 4th document:', documents[3].page_content[0:50], documents[3].metadata['source'])
print('Total number of tokens:',num_tokens_from_string(concatenate_page_contents(documents, ' ')+query+prompt_template, "gpt-3.5-turbo"))
print('Prompt and query number of tokens:',num_tokens_from_string(query+prompt_template, "gpt-3.5-turbo"))

Prompt template: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}
User query: What is the api endpoint I need to call to submit an invoice?
Number of relevant documents: 1
Start of first document: For example, the NDIS has an overnight adjudicatio https://docs.lanternpay.com/docs/provider-api.html
Total number of tokens: 41084
Prompt and query number of tokens: 57


#### Manually retrieving the number of relevant documents 

I"m wondering why this single document has so many tokens

Let's retrieve relevant documents without langchain but just using the ChromaDB api

In [ ]:
 # This is what the Chroma retriever is doing, it is querying the Chroma DB
 # /workspaces/IsabelleLangchain/.venv/lib/python3.10/site-packages/langchain/vectorstores/chroma.py

coll = chromaClient.get_collection(name='LANTERNPAY_COLLECTION', embedding_function=OpenAIEmbeddings().embed_documents)
resultQuery = coll.query(query_texts=[query], n_results=2)
pprint(resultQuery)

In [35]:
qa.dict()

{'memory': None,
 'verbose': True,
 'tags': None,
 'metadata': None,
 'combine_documents_chain': {'memory': None,
  'verbose': False,
  'tags': None,
  'metadata': None,
  'input_key': 'input_documents',
  'output_key': 'output_text',
  'llm_chain': {'memory': None,
   'verbose': False,
   'tags': None,
   'metadata': None,
   'prompt': {'input_variables': ['question', 'context'],
    'output_parser': None,
    'partial_variables': {},
    'messages': [{'prompt': {'input_variables': ['context'],
       'output_parser': None,
       'partial_variables': {},
       'template': "Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}",
       'template_format': 'f-string',
       'validate_template': True,
       '_type': 'prompt'},
      'additional_kwargs': {}},
     {'prompt': {'input_variables': ['question'],
       'output_parser': None,
       'partial

In [ ]:
query = "How do I send an invoice to TAC program?"
# result = qa.run(query)
result = qa({"query": query})

pprint(result)

In [17]:
# debugging the score of the search

chromaDb.similarity_search_with_score(query)

[(Document(page_content='Welcome to the LanternPay by HICAPS Developer Hub. Here you’ll find information & API references to help you integrate with LanternPay as quickly as possible. If you need a hand, email integrations@hicaps.com.au and one of our friendly team will get back to you as soon as possible!\nIf you are interested in becoming an integration partner and would like to get access to our sandbox environment, please fill out this access request form and one of our team will be in touch.', metadata={'source': 'https://docs.lanternpay.com/'}),
  0.31823283433914185),
 (Document(page_content='Program API\nOverview\nThe Program API is used to communicate between LanternPay and a Program (i.e. scheme) for the purpose of determining the funding result for claims submitted by billers. To facilitate this communication, the Program API comprises a set of webhooks and RESTful API resources for consumption.\nOnboarding\nBillers register with LanternPay. When a biller indicates a desire 

In [18]:
from pprint import pprint
# https://realpython.com/python-pretty-print/
pprint(qa.json())

('{"memory": null, "verbose": true, "tags": null, "metadata": null, '
 '"combine_documents_chain": {"memory": null, "verbose": false, "tags": null, '
 '"metadata": null, "input_key": "input_documents", "output_key": '
 '"output_text", "llm_chain": {"memory": null, "verbose": false, "tags": null, '
 '"metadata": null, "prompt": {"input_variables": ["context", "question"], '
 '"output_parser": null, "partial_variables": {}, "messages": [{"prompt": '
 '{"input_variables": ["context"], "output_parser": null, "partial_variables": '
 '{}, "template": "Use the following pieces of context to answer the users '
 "question. \\nIf you don't know the answer, just say that you don't know, "
 'don\'t try to make up an answer.\\n----------------\\n{context}", '
 '"template_format": "f-string", "validate_template": true, "_type": '
 '"prompt"}, "additional_kwargs": {}}, {"prompt": {"input_variables": '
 '["question"], "output_parser": null, "partial_variables": {}, "template": '
 '"{question}", "templ

In [6]:
pprint(qa.combine_documents_chain.llm_chain.prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}", template_format='f-string', validate_template=True), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='{question}', template_format='f-string', validate_template=True), additional_kwargs={})]


In [11]:

print(result)

{'query': 'What is Lanternpay?', 'result': 'LanternPay is a platform provided by HICAPS that allows for secure and efficient payment processing, claims management, and reimbursement for healthcare services. It offers a range of APIs and integration options for developers to integrate LanternPay into their systems.', 'source_documents': [Document(page_content='Welcome to the LanternPay by HICAPS Developer Hub. Here you’ll find information & API references to help you integrate with LanternPay as quickly as possible. If you need a hand, email integrations@hicaps.com.au and one of our friendly team will get back to you as soon as possible!\nIf you are interested in becoming an integration partner and would like to get access to our sandbox environment, please fill out this access request form and one of our team will be in touch.', metadata={'source': 'https://docs.lanternpay.com/'})]}


In [5]:
from langchain.agents import Tool

def qaTool(query):
    return qa({"query": query})

name = """Lanternpay By Hicaps Developer Hub"""

description = """Useful for when you need to answer questions about the Provider API that provides resources for service providers and PMS software to connect to Lanternpay by Hicaps, or answer questions about the Program API that provides resources for Programs, or Schemes to connect to Lanternpay by Hicaps."""

lanternpay_tool = Tool(
    name=name,
    func=qaTool,
    description=description,
)


## Make chatbot (agent) with tools


### custom output parser

In [6]:
import re
from typing import Union

from langchain.agents.agent import AgentOutputParser
# from langchain.agents.conversational.prompt import FORMAT_INSTRUCTIONS
from langchain.schema import AgentAction, AgentFinish, OutputParserException

FORMAT_INSTRUCTIONS = """To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, you MUST use the format:

```
Thought: Do I need to use a tool? No
{ai_prefix}: [your response here]
```"""


class ConvoOutputParser2(AgentOutputParser):
    ai_prefix: str = "AI"

    def get_format_instructions(self) -> str:
        return FORMAT_INSTRUCTIONS

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        
        print('***********************************************************')
        print(f'{text}')
        print(f'output AgentFinish:{text.split(f"{self.ai_prefix}:")[-1].strip()}')
        
        if f"{self.ai_prefix}:" in text:
            return AgentFinish(
                {"output": text.split(f"{self.ai_prefix}:")[-1].strip()}, text
            )
        regex = r"Action: (.*?)[\n]*Action Input: (.*)"
        match = re.search(regex, text)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{text}`")
        action = match.group(1)
        action_input = match.group(2)
        return AgentAction(action.strip(), action_input.strip(" ").strip('"'), text)

    @property
    def _type(self) -> str:
        return "conversational"

### agent executor with tools

In [17]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, load_tools, AgentType

llm = ChatOpenAI()

tools = load_tools([
    'wikipedia', 
    'llm-math', 
    'google-search'
], llm=llm)


tools.append(lanternpay_tool)

memory = ConversationBufferMemory(memory_key='chat_history')

agentExecutor = initialize_agent(
    tools, 
    llm, 
    # agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    # output_parser=ConvoOutputParser2.parse
)

In [24]:
# pprint(agent.agent.llm_chain.prompt.template)

In [28]:
agentExecutor.tools

[WikipediaQueryRun(name='Wikipedia', description='A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/workspaces/IsabelleLangchain/.venv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='Calculator', description='Useful for when you need to answer questions about math.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method Chain.run of LLMMathChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, 

In [29]:
allowed_tools=agentExecutor.agent.get_allowed_tools()
tools=agentExecutor.tools
print(set(allowed_tools))
print(set([tool.name for tool in tools]))

{'Calculator', 'Wikipedia', 'google_search', 'Lanternpay By Hicaps Developer Hub'}
{'Calculator', 'Wikipedia', 'google_search', 'Lanternpay By Hicaps Developer Hub'}


### Calling agent executor with the query

In [19]:
response = agentExecutor.run('What is the program API?')



> Entering new AgentExecutor chain...
Isabelle Agent Plan Full Input: {'input': 'What is the program API?', 'chat_history': 'Human: What is the program API?\nAI: The Program API is a tool provided by Lanternpay by Hicaps that allows Programs or Schemes to connect to the Lanternpay platform and interact with various features and functions of the system.', 'agent_scratchpad': '', 'stop': ['Observation:']}
Isabelle Agent Plan Full OUtput: The Program API is a part of the Lanternpay by Hicaps Developer Hub. It provides resources for Programs or Schemes to connect to Lanternpay by Hicaps. To learn more details about the Program API, we can refer to the Lanternpay By Hicaps Developer Hub.

Thought: I should use the Lanternpay By Hicaps Developer Hub to find information about the Program API.
Action:
```
{
  "action": "Lanternpay By Hicaps Developer Hub",
  "action_input": "Program API"
}
```
The Program API is a part of the Lanternpay by Hicaps Developer Hub. It provides resources for Prog

NameError: name 'pprint' is not defined

In [21]:
from pprint import pprint
pprint(response)

('The Program API is used to communicate between LanternPay and a Program '
 '(scheme) for determining funding results for claims submitted by billers. It '
 'includes webhooks and RESTful API resources for consumption.')


In [23]:
agentExecutor.run('What is the provider API?')



> Entering new AgentExecutor chain...
Isabelle Agent Plan Full Input: {'input': 'What is the provider API?', 'chat_history': 'Human: What is the program API?\nAI: The Program API is a tool provided by Lanternpay by Hicaps that allows Programs or Schemes to connect to the Lanternpay platform and interact with various features and functions of the system.\nHuman: What is the program API?\nAI: The Program API is used to communicate between LanternPay and a Program (scheme) for determining funding results for claims submitted by billers. It includes webhooks and RESTful API resources for consumption.', 'agent_scratchpad': '', 'stop': ['Observation:']}
Isabelle Agent Plan Full OUtput: The Provider API is a part of the Lanternpay by Hicaps Developer Hub. It provides resources for service providers and PMS (Practice Management Software) to connect to Lanternpay by Hicaps. It allows service providers to access various functionalities, such as creating claims, managing payments, checking clai

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 8106 tokens. Please reduce the length of the messages.

## somen random stuff


In [ ]:

from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

query_engine = index.as_query_engine()
tool_config = IndexToolConfig(
    query_engine=query_engine, 
    name=f"Financial Report",
    description=f"useful for when you want to answer queries about the Apple financial report",
    tool_kwargs={"return_direct": True}
)

memory = ConversationBufferMemory(memory_key="chat_history")
llm=ChatOpenAI(temperature=0.2,model_name='gpt-3.5-turbo')
agent_chain = create_llama_chat_agent(
    toolkit,
    llm,
    memory=memory,
    verbose=True
)

# random something

In [46]:
from langchain.agents.structured_chat.base import StructuredChatAgent
from langchain.agents.agent import AgentExecutor
from langchain.chat_models import AzureChatOpenAI
from langchain.tools.python.tool import PythonAstREPLTool
from langchain.schema import AgentAction


tools = [PythonAstREPLTool()]

llm = ChatOpenAI(temperature=0.)

agent_obj = StructuredChatAgent.from_llm_and_tools(
    llm=llm,
    tools=tools,
        memory=memory
)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent_obj,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True
)

response = agent_executor({"input": "Tell me something interesting using your python tool."})



> Entering new  chain...
Sure! How about I generate a random number for you using Python? Would you like that?

> Finished chain.


In [47]:
response

{'input': 'Tell me something interesting using your python tool.',
 'output': 'Sure! How about I generate a random number for you using Python? Would you like that?',
 'intermediate_steps': []}